# imp

> see the python documentation about [programmatic imports](https://docs.python.org/3/library/importlib.html#importing-programmatically)

In [ ]:
#| default_exp imp

In [1]:
#| hide
from nbdev.showdoc import *

In [1]:
#| export
import sys, types, inspect, importlib, warnings
from importlib.util import (LazyLoader, find_spec, module_from_spec)
from importlib.abc import Loader
from importlib.machinery import ModuleSpec

from dataclasses import dataclass, field
from typing import Optional, Dict, Any, TypeAlias, Union, ClassVar, List, Tuple

In [2]:
#| export
@dataclass
class GlobalImport:
    def __enter__(self):
        return self
    
    def __call__(self):    
        currentframe = inspect.currentframe()
        outerframes  = inspect.getouterframes(currentframe)
        outerframe   = outerframes[1].frame
        self.collector = inspect.getargvalues(outerframe).locals

    def __exit__(self, *args):
        globals().update(self.collector)

In [3]:
#| export
Module: TypeAlias = types.ModuleType

Loader: TypeAlias = importlib.abc.Loader

ModuleSpec: TypeAlias = importlib.machinery.ModuleSpec

Fallbacks: TypeAlias = Dict[str, Any]

VariableDict: TypeAlias = Dict[str, Any]

In [4]:
#| export
def modjoin(*parts: str) -> str:
    """
    Join module name parts into a valid module path.

    Parameters
    ----------
    *parts : str
        Name parts of the module to be joined.

    Returns
    -------
    str
        Valid module path.
    """
    return ('.'.join(parts)).rstrip('.')

def is_mod(module: Any) -> bool:
    """
    Check if a given object is a module.

    Parameters
    ----------
    module : Any
        Object to be checked.

    Returns
    -------
    bool
        True if the object is a module, False otherwise.
    """
    return isinstance(module, Module)

def is_mod_imp(name: str) -> bool:
    """
    Check if a given module name is in the system module list.

    Parameters
    ----------
    name : str
        Name of the module to be checked.

    Returns
    -------
    bool
        True if the module is in the system module list, False otherwise.
    """
    return name in sys.modules

def is_var_imp(name: str) -> bool:
    """
    Check if a given variable name exists in the global namespace.

    Parameters
    ----------
    name : str
        Name of the variable to be checked.

    Returns
    -------
    bool
        True if the variable exists in the global namespace, False otherwise.
    """
    return name in globals()

def is_mod_or_var(name: str):
    """
    Check if a given name exists either as a module or a variable.

    Parameters
    ----------
    name : str
        Name to be checked.

    Returns
    -------
    bool
        True if the name exists either as a module or a variable, False otherwise.
    """
    return is_var_imp(name) or is_mod_imp(name)

def is_mod_avail(name: str) -> bool:
    """
    Check if a given module name is available for import.

    Parameters
    ----------
    name : str
        Name of the module to be checked.

    Returns
    -------
    bool
        True if the module is available for import, False otherwise.
    """
    if is_mod_imp(name): return True
    elif find_spec(name) is not None: return True
    return False

def loader_from_spec(spec: ModuleSpec, lazy: bool = False) -> Loader:
    """
    Get a loader from a given module specification.

    Parameters
    ----------
    spec : ModuleSpec
        Module specification from which to get the loader.
    lazy : bool, optional
        If True, return a LazyLoader; if False, return the original loader.

    Returns
    -------
    Loader
        Loader for the module.
    """
    loader = spec.loader
    if lazy: loader = LazyLoader(loader)
    return loader

def module_from_str(
    name: str, 
    lazy: bool = False, 
    alias: Optional[str] = None,
    inject: Optional[bool] = True,
    inject_both: Optional[bool] = False,
) -> Module:
    """
    Import a module from a given string name.

    Parameters
    ----------
    name : str
        Name of the module to import.
    lazy : bool, optional
        If True, use lazy import; if False, use regular import.
    alias : str, optional
        Alias to use for the module.
    inject : bool, optional
        If True, inject the module into sys.modules.
    inject_both : bool, optional
        If True, inject the module into sys.modules with both its original name and alias.

    Returns
    -------
    Module
        The imported module.
    """
    spec = find_spec(name)    
    loader = loader_from_spec(spec, lazy=lazy)
    
    module = module_from_spec(spec)    
    modname = name if alias is None else alias
    
    if inject_both: inject = True
    if inject:
        sys.modules[modname] = module
    if inject_both:
        sys.modules[name] = module

    loader.exec_module(module)
    return module

def getmodule(
    name: str, lazy: Optional[bool] = False, 
    alias: Optional[str] = None, 
    inject: Optional[bool] = True,
    inject_both: Optional[bool] = False,
) -> Module:
    """
    Get a module by name, importing it if necessary.

    Parameters
    ----------
    name : str
        Name of the module to get.
    lazy : bool, optional
        If True, use lazy import; if False, use regular import.
    alias : str, optional
        Alias to use for the module.
    inject : bool, optional
        If True, inject the module into sys.modules.
    inject_both : bool, optional
        If True, inject the module into sys.modules with both its original name and alias.

    Returns
    -------
    Module
        The requested module.

    Raises
    ------
    ModuleNotFoundError
        If the module cannot be found.
    """
    if is_mod_avail(name):
        return module_from_str(name, lazy, alias, inject, inject_both)
    raise ModuleNotFoundError


In [5]:
@dataclass
class ImpSpec(ModuleSpec):
    """
    A class to represent an import specification.

    Attributes
    ----------
    name : str
        The name of the module to import.
    nick : Optional[str]
        An optional nickname for the module.
    lazy : Optional[bool]
        Whether to perform a lazy import. Defaults to True.
    subspecs : Optional[List[ImpSubSpec]]
        A list of sub-specifications for the import.
    fallbacks : Optional[Fallbacks]
        A mapping of fallback values for each item.
    origin : ClassVar[Optional[Any]]
        The origin of the module. Defaults to None.
    is_package : ClassVar[Optional[bool]]
        Whether the module is a package. Defaults to False.
    submodule_search_locations : ClassVar[Optional[List[str]]]
        Search locations for the submodules. Defaults to an empty list.
    """
    ...


In [6]:
#| export
@dataclass
class ImpItem:
    """
    A class to represent an item in the import specification.

    Attributes
    ----------
    item : str
        The item to be imported.
    name : Optional[str]
        The name of the item to be imported. Derived from `item` during post-initialization.
    nick : Optional[str]
        The nickname of the item to be imported. Derived from `item` during post-initialization.
    """
    item: str = field(repr=False, default='')
    name: Optional[str] = field(init=False, repr=True)
    nick: Optional[str] = field(init=False, repr=True)

    def __post_init__(self):
        name, _, nick = self.item.strip().partition(' as ')
        self.name = name
        self.nick = nick        
        return self
    
    def __eq__(self, other: Any) -> bool:
        if isinstance(other, str):
            return self.varname == other
        elif isinstance(other, ImpItem):
            return self.varname == other.varname
        return False
    
    @property
    def has_alias(self) -> bool:
        return bool(self.nick)
    
    @property
    def varname(self) -> bool:
        varname = self.name if not self.nick else self.nick
        return varname

In [7]:
#| export
@dataclass
class ImpSubSpec:
    """
    A class to represent a sub-specification of the import.

    Attributes
    ----------
    name : str
        The name of the module from which items are imported.
    stub : str
        The sub-module path.
    items : List[ImpItem]
        A list of items to import.
    fallbacks : Optional[Fallbacks]
        A mapping of fallback values for each item.
    """
    name: str
    stub: str
    items: List[ImpItem] = field(default_factory=list)
    fallbacks: Optional[Fallbacks] = field(default_factory=dict, repr=False)

    def __post_init__(self):
        """
        Override the post-initialization method to create fallbacks for the instance.
        """
        fallbacks = self.make_fallbacks()        
        self.fallbacks = fallbacks

    def items_from_str(self, items: str) -> 'ImpSubSpec':
        """
        Converts a string of comma-separated items into a list of ImpItems.
        
        Parameters
        ----------
        items : str
            A string of comma-separated items.
        
        Returns
        -------
        self : 'ImpSubSpec'
            The current instance of the ImpSubSpec.
        """
        items = [ImpItem(item) for item in items.split(',')]
        return self

    @classmethod
    def from_str(cls, items: str) -> 'ImpSubSpec':
        mods, _, items = items.strip().partition(' import ')        
        _from, _, mods = mods.partition('from ')        
        name, *stub = mods.split('.')
        stub = '.'.join(stub)
        items = [ImpItem(item) for item in items.split(',')]
        return cls(name, stub, items)
    
    @property
    def path(self) -> str:
        """
        Returns the full path of the module by joining the name and stub.

        Returns
        -------
        str
            The full path of the module.
        """
        return modjoin(self.name, self.stub)
    
    def update_fallbacks(self, fallbacks: Optional[Fallbacks] = None) -> Fallbacks:
        fallbacks = fallbacks or self.default_fallbacks()
        fallbacks = getattr(self, 'fallbacks', fallbacks)
        self.fallbacks.update(fallbacks)
        return self
    
    def make_fallbacks(self) -> Fallbacks:
        fallbacks = self.default_fallbacks()
        fallbacks.update(self.fallbacks)
        fallbacks = {k: v for k, v in fallbacks.items() if k in self.items}
        return fallbacks
    
    def default_fallbacks(self):
        fallbacks = dict()
        for imp_item in self.items:
            varname = imp_item.varname
            fallbacks[varname] = None
        return fallbacks

    def make_failsafe(self, updates: Optional[VariableDict] = dict()) -> VariableDict:
        updates = getattr(self, 'updates', (updates or dict()))
        for name, fallback in self.fallbacks.items():
            if name not in updates and not is_mod_or_var(name):
                updates[name] = fallback
        return updates
    
    def getitem(self, item: ImpItem) -> Tuple[str, Any]:
        """
        Attempts to import an item from the module. 

        Parameters
        ----------
        item : ImpItem
            The item to be imported.
        
        Returns
        -------
        Tuple[str, Any]
            The name of the item and its value, or None if import fails.
        """
        varname = item.varname                
        try:                
            submod = getmodule(self.path, inject=False)
            varval = getattr(submod, item.name, None)
            return varname, varval
        except ImportError or ModuleNotFoundError:
            warnings.warn(f'Could not import {varname} from  {self.path}')
            return varname, None
    
    def fetch(self) -> VariableDict:
        """
        Fetches all the items in the import specification.

        Returns
        -------
        VariableDict
            A dictionary mapping the names of the items to their values.
        """
        updates = dict()
        for imp_item in self.items:
            varname, varval = self.getitem(imp_item)

            updates[varname] = varval
            if varval is None:
                updates[varname] = self.fallbacks.get(varname, None)
        return updates
    
    def get_updates(self) -> VariableDict:
        """
        Fetches all updates.

        Returns
        -------
        VariableDict
            A dictionary mapping the names of the items to their updated values.
        """
        updates = self.fetch()
        updates = self.make_failsafe(updates)
        return updates
    
    def update_globals(self):
        """
        Updates the global namespace with the fetched updates.
        
        Returns
        -------
        self : 'ImpSubSpec'
            The current instance of the ImpSubSpec.
        """
        updates = self.get_updates()        
        globals().update(updates)
        return self

In [8]:
#| eval: False
ImpSubSpec.from_str('from rich.tree import Tree as rich_tree')

ImpSubSpec(name='rich', stub='tree', items=[ImpItem(name='Tree', nick='rich_tree')])

In [9]:
#| eval: False
ImpSubSpec('rich', '', [ImpItem('get_console')])

ImpSubSpec(name='rich', stub='', items=[ImpItem(name='get_console', nick='')])

In [10]:
#| eval: False
ImpSubSpec.from_str('from rich import get_console')

ImpSubSpec(name='rich', stub='', items=[ImpItem(name='get_console', nick='')])

In [11]:
#| export
@dataclass
class ImpSpec(ModuleSpec):
    """
    A class to represent an import specification.

    Attributes
    ----------
    name : str
        The name of the module to import.
    nick : Optional[str]
        An optional nickname for the module.
    lazy : Optional[bool]
        Whether to perform a lazy import. Defaults to True.
    subspecs : Optional[List[ImpSubSpec]]
        A list of sub-specifications for the import.
    fallbacks : Optional[Fallbacks]
        A mapping of fallback values for each item.
    origin : ClassVar[Optional[Any]]
        The origin of the module. Defaults to None.
    is_package : ClassVar[Optional[bool]]
        Whether the module is a package. Defaults to False.
    submodule_search_locations : ClassVar[Optional[List[str]]]
        Search locations for the submodules. Defaults to an empty list.
    """
    name: str
    nick: Optional[str] = field(default='')
    lazy: Optional[bool] = field(default=True)
    subspecs: Optional[List[ImpSubSpec]] = field(default_factory=list, repr=False)
    fallbacks: Optional[Fallbacks] = field(default_factory=dict, repr=False)

    origin: ClassVar[Optional[Any]] = field(default=None)
    is_package: ClassVar[Optional[bool]] = field(default=False)
    submodule_search_locations: ClassVar[Optional[List[str]]] = field(default=list)

    def _get_loader(self, spec:ModuleSpec) -> Loader:
        return loader_from_spec(spec, lazy=self.lazy)

    def __post_init__(self):
        spec = find_spec(self.name)
        loader = self._get_loader(spec)
        self.submodule_search_locations = spec.submodule_search_locations
        for subspec in self.subspecs:
            subspec.update_fallbacks(self.fallbacks)
            
        super().__init__(self.name, loader)
        
    def _imp_main(self):
        try:
            mod = getmodule(self.name, lazy=self.lazy, alias=self.nick, inject_both=True)
            self._module = mod

        except ImportError or ModuleNotFoundError:
            sys.modules[self.name] = self.fallbacks.get(self.name, None)
            sys.modules[self.nick] = self.fallbacks.get(self.nick, None)
            warnings.warn(f'Could not import {self.name}')
            return
        
    def _imp_subs(self):
        updates = dict()
        for subspec in self.subspecs:
            updates.update(subspec.get_updates())

        for name, fallback in self.fallbacks.items():
            if name not in updates and not is_mod_or_var(name):
                updates[name] = fallback

        globals().update(updates)

In [12]:
#| export
@dataclass
class Imp:
    """
    Import Module class.
    
    This class facilitates dynamic import of modules and their attributes.
    
    Parameters
    ----------
    name : str
        The name of the module to import.
    nick : Optional[str], default=None
        The alias of the module to import.
    subspecs : Optional[List[ImpSubSpec]], default=list()
        A list of `ImpSubSpec` objects representing additional specifications for import.
    fallbacks : Optional[Dict[str, Any]], default=dict()
        A dictionary of fallback values for import failures.
    lazy : Optional[bool], default=True
        Whether or not to use lazy import.
    delay: Optional[bool], default=False
        Whether or not to delay import.

    Attributes
    ----------
    _module : Module
        The imported module.
    _spec : ImpSpec
        The specification used for import.

    Methods
    -------
    __enter__():
        Enter the runtime context for `Imp`.
    __exit__(exc_type, exc_value, traceback):
        Exit the runtime context for `Imp`.
    __getitem__(key: str) -> Any:
        Get an attribute from the imported module.
    """
    name: str
    nick: Optional[str] = field(default=None)
    subspecs: Optional[List[ImpSubSpec]] = field(default_factory=list, repr=False)
    fallbacks: Optional[Dict[str, Any]] = field(default_factory=dict, repr=False)
    lazy: Optional[bool] = field(default=True)
    delay: Optional[bool] = field(default=False)

    _module: Module = field(init=False, repr=False, default=None)
    _spec: ImpSpec = field(init=False, repr=False, default=None)

    def load(self):
        self._spec = ImpSpec(self.name, self.nick, self.lazy, self.subspecs, self.fallbacks)
        self._spec._imp_main()
        self._spec._imp_subs()
        self._module = sys.modules[self.name]

    def __post_init__(self):
        if not self.delay:
            self.load()

    
    
    def __enter__(self):
        """
        Enter the runtime context for `Imp`.

        This method is called when the `with` statement is used with an `Imp` instance.
        """
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        """
        Exit the runtime context for `Imp`.

        This method is called when exiting the `with` statement.

        Parameters
        ----------
        exc_type : Exception type
            The type of exception raised within the `with` block, if any.
        exc_value : Exception
            The instance of exception raised within the `with` block, if any.
        traceback : Traceback
            The traceback object encapsulating the call stack at the point where the exception was raised, if any.
        """
        # If you want to handle exceptions that happen inside the `with` block,
        # you can add the logic here.
        pass

    def __getitem__(self, key: str) -> Any:
        """
        Get an attribute from the imported module.

        Parameters
        ----------
        key : str
            The name of the attribute.

        Returns
        -------
        Any
            The attribute from the imported module.
        """
        return getattr(self._module, key, None)


In [13]:
#| eval: False
imp_rich = Imp(
    'rich', 'rc'
    subspecs=[
        ImpSubSpec.from_str('from rich.tree import Tree as rich_tree'),
        ImpSubSpec.from_str('from rich.text import Text'),
        ImpSubSpec.from_str('from rich.markup import espace'),
        ImpSubSpec.from_str('from rich.filesize import decimal'),
        ImpSubSpec.from_str('from rich.filesize import Console'),
        ImpSubSpec.from_str('from rich.progress import Progress'),
        # ImpSubSpec.from_str('from rich import get_console')
        ImpSubSpec('rich', '', [ImpItem('get_console')])
    ],
    fallbacks={
        'rich_tree': Any,
        'Text': Any,
    },
    delay=True
)

In [14]:
#| eval: False
Text

NameError: name 'Text' is not defined

In [15]:
#| eval: False
imp_rich.load()

In [16]:
#| export
Text

rich.text.Text

In [16]:
#| hide
import nbdev; nbdev.nbdev_export()